# Twitter Analysis Pipeline

![](https://www.evernote.com/l/AAFtiWq4lUNNiqyswF72dUV3f_7Geq_TcykB/image.png)

In [1]:
! pip install boto3 pymongo twitter --quiet

In [2]:
import lib 

## Write Tweets to S3

### Create a Tweet Iterator

In [3]:
consumer_key    = "MkzCxkb2dEjT0lJakO8aaYyNy"
consumer_secret = "50gIenCSmtC6OVVhfr11KZ7RXuflqL5P4yIG6qWPft0lzg7yPS"
token           = "494647099-hRiZkTdJDnqj3wGEfBv8rC8vhouzwhLc1hqvn9Wb"
token_secret    = "IMqQHCjSyaBiuaR1xJStQaRiNM3Z6oiNcrXvSRzXckO4g"
bounding_box    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

tweet_iterator  = lib.create_tweet_iterator(token, 
                                            token_secret,
                                            consumer_key,
                                            consumer_secret,
                                            bounding_box)

In [4]:
tweets = [next(tweet_iterator) for _ in range(50)]

In [5]:
tweets_2 = []
for _ in range(50):
    tweets_2.append(next(tweet_iterator))

### Write list of tweets to JSON file on disk

In [6]:
import json 

username = 'chillsimmons'

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)
    
filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets_2, outfile)    

### Write Tweet Files to S3, Step 1 - Create a Boto Client to S3

https://boto3.readthedocs.io

In [7]:
s3_client = lib.create_boto_client()

In [8]:
from os import listdir

In [9]:
current_directory = listdir()
current_directory = [file for file in current_directory if 'tweets-chillsimmons' in file]
current_directory

['tweets-chillsimmons-2018-07-27_00-53-46-422094.json',
 'tweets-chillsimmons-2018-07-27_00-53-46-401750.json']

In [10]:
from os import rename

### Write Tweet Files to S3, Step 2 - Write a file to S3

In [11]:
S3_BUCKET = 'uclax-data-science'

for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [12]:
current_directory = listdir()
current_directory = [file for file in current_directory if 'tweets-chillsimmons' in file]
current_directory

['tweets-chillsimmons-2018-07-27_00-53-46-401750-processed.json',
 'tweets-chillsimmons-2018-07-27_00-53-46-422094-processed.json']

### List Files on S3

In [13]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-chillsimmons' in file]
s3_files

['tweets-chillsimmons-2018-07-27_00-53-46-401750.json',
 'tweets-chillsimmons-2018-07-27_00-53-46-422094.json']

## Write Tweets to Mongo

### Read an object from S3

In [14]:
S3_BUCKET = 'uclax-data-science'
key = s3_files[1]

tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)

### Write Tweets to Mongo

In [15]:
from pymongo.errors import DuplicateKeyError

In [16]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

In [17]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee04376af00ba09c5a1') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a2') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a3') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a4') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a5') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a6') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a7') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b5a6ee14376af00ba09c5a8') }
E11000 duplicate key error collection: twitter.tweets in

In [18]:
for key in s3_files[2:]:
    for tweet in tweets_from_s3:
        try:
            collection_client.insert_one(tweet)
        except DuplicateKeyError as e:
            print(e)

## One More Time

In [19]:
tweets = [next(tweet_iterator) for _ in range(50)]

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)

In [20]:
current_directory = listdir()
current_directory = [file for file in current_directory 
                     if ('tweets-chillsimmons' in file) and ('processed' not in file)]
current_directory

['tweets-chillsimmons-2018-07-27_01-06-45-304032.json']

In [21]:
for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [22]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-chillsimmons' in file]
s3_files

['tweets-chillsimmons-2018-07-27_00-53-46-401750.json',
 'tweets-chillsimmons-2018-07-27_00-53-46-422094.json',
 'tweets-chillsimmons-2018-07-27_01-06-45-304032.json']

In [ ]:
s3_files[-1]

In [ ]:
key = s3_files[-1]

In [ ]:
tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)

In [ ]:
collection_client.insert_many(tweets_from_s3)

## Automate It

In [ ]:
from datetime import datetime

In [ ]:
TOKEN           = "12270642-8ggOVEjcQ7DAP2VCCZbXYoRtiOf0rngzrsyb82h7k"
TOKEN_SECRET    = "85KnJJBbdvhZyGhOVLPwk47x2IAHaLEFGhNJdrHXo1Tv8"
CONSUMER_KEY    = "Jt9H3ve228Tl8yafi4Ip8Iwbw"
CONSUMER_SECRET = "xXxkq7QM1D7K1w0tFKRIYYviS2EQ288jzbbiwzEYYRQrMyvCDN"
BOUNDING_BOX    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

DATABASE        = 'twitter'
COLLECTION      = 'tweets'
USERNAME        = 'joshua'

S3_BUCKET       = 'uclax-data-science'

In [ ]:
tweet_iterator    = lib.create_tweet_iterator(TOKEN, TOKEN_SECRET, 
                                              CONSUMER_KEY, CONSUMER_SECRET, BOUNDING_BOX)
s3_client         = lib.create_boto_client()
collection_client = lib.create_mongo_client_to_database_collection(DATABASE, COLLECTION)

In [ ]:
def timestamp():
    now = datetime.now().strftime('%D %H:%M:%S')
    print(now, end=' | ')
    print('Collecting Tweets', end=' | ')
    
def collect_tweets(n=50):
    tweets = [next(tweet_iterator) for _ in range(n)]
    print('{} Tweets'.format(n), end=' | ')
    return tweets
    
def write_to_disk(tweets):
    filename = lib.create_timestamped_filename(USERNAME) 
    with open(filename, 'w') as outfile:
        json.dump(tweets, outfile)    
    print('Written to Disk', end=' | ')
    return filename
    
def write_to_S3(filename):   
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)
    print('Written to S3', end=' | ')
    
def insert_to_mongo(key):
    tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)    
    collection_client.insert_many(tweets_from_s3)
    print('Inserted to Mongo')

In [ ]:
while True:
    timestamp()
    tweets = collect_tweets()
    filename = write_to_disk(tweets)
    write_to_S3(filename)
    insert_to_mongo(filename)